In [1]:
from __future__ import division, print_function
%matplotlib inline

In [2]:
import sympy
from  sympy import Matrix, eye, symbols, sin, cos, zeros, sqrt
from sympy.physics.mechanics import *
from IPython.display import display
sympy.init_printing(use_latex='mathjax')


# Quaternion Math

In [3]:
def expq(n):
    n *= 0.5
    nNorm = n.norm()
    qn = Matrix([cos(nNorm),n/nNorm*sin(nNorm)])
    return qn

def quat2dcm(q):
    """
    Convert quaternion to DCM
    """
    
    # Extract components
    w = q[0]
    x = q[1]
    y = q[2]
    z = q[3]
    
    # Reduce repeated calculations
    ww = w*w
    xx = x*x
    yy = y*y
    zz = z*z  
    wx = w*x
    wy = w*y
    wz = w*z
    xy = x*y
    xz = x*z
    yz = y*z
    
    # Build Direction Cosine Matrix (DCM)   
    dcm = Matrix([
        [ww + xx - yy - zz,       2*(xy - wz),       2*(xz + wy)],
        [      2*(xy + wz), ww - xx + yy - zz,       2*(yz - wx)],
        [      2*(xz - wy),       2*(yz + wx), ww - xx - yy + zz]
    ])
    return dcm

def dcm2quat(dcm):
    """
    Determine quaternion corresponding to dcm using
    the stanley method. 
    
    Flips sign to always return shortest path quaterion
    so w >= 0
    
    Converts the 3x3 DCM into the quaterion where the 
    first component is the real part
    """
    
    tr = Matrix.trace(dcm)
    
    w = 0.25*(1+tr)
    x = 0.25*(1+2*dcm[0,0]-tr)
    y = 0.25*(1+2*dcm[1,1]-tr)
    z = 0.25*(1+2*dcm[2,2]-tr)
    
    #kMax = np.argmax([w,x,y,z])
    
    kMax = 0
    
    if kMax == 0:
        w = sqrt(w)
        x = 0.25*(dcm[1,2]-dcm[2,1])/w
        y = 0.25*(dcm[2,0]-dcm[0,2])/w
        z = 0.25*(dcm[0,1]-dcm[1,0])/w
    
    elif kMax == 1:
        x = sqrt(x)
        w = 0.25*(dcm[1,2]-dcm[2,1])/x
        if w<0:
            x = -x
            w = -w
        y = 0.25*(dcm[0,1]+dcm[1,0])/x
        z = 0.25*(dcm[2,0]+dcm[0,2])/x
        
    elif kMax == 2:
        y = sqrt(y)
        w = 0.25*(dcm[2,0]-dcm[0,2])/y
        if w<0:
            y = -y
            w = -w
        x = 0.25*(dcm[0,1]+dcm[1,0])/y
        z = 0.25*(dcm[1,2]+dcm[2,1])/y
        
    elif kMax == 3:
        z = sqrt(z)
        w = 0.25*(dcm[0,1]-dcm[1,0])/z
        if w<0:
            z = -z
            w = -w
        x = 0.25*(dcm[2,0]+dcm[0,2])/z
        y = 0.25*(dcm[1,2]+dcm[2,1])/z
        
    q = Matrix([w,x,y,z])
    
    return q

def skew3(v):
    vx,vy,vz = v
    out = Matrix([[  0, -vz,   vy],
                  [ vz,   0,  -vx],
                  [-vy,  vx,    0]])
    return out

def skew4Left(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w],
        ])        
    return out

def skew4Right(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w,  z, -y],
            [y, -z,  w,  x],
            [z,  y, -x,  w],
        ])      
    return out


def quatConj(q):
    q_out = Matrix(q[:])
    q_out = q_out.T*sympy.diag(1,-1,-1,-1)
    q_out = q_out.T

    return q_out

def qRot(q,v):
    qPrime = quatConj(q)
    v = Matrix.vstack(zeros(1),v)
    vout = skew4Left(q)*skew4Right(qPrime)*v
    return Matrix(vout[1:])

# Setup

In [4]:
qw,qx,qy,qz = symbols('q_w q_x q_y q_z')
t = symbols('T')
mx,my,mz = symbols('m_x m_y m_z') # measured 
ax,ay,az = symbols('a_x a_y a_z') # measured
wx,wy,wz = symbols('w_x w_y w_z') # measured
g = symbols('g')
bx,by,bz = symbols('b_x b_y b_z') 

q_toLfromB = Matrix([qw,qx,qy,qz])

magMeas = Matrix([mx,my,mz]) 
accelMeas = Matrix([ax,ay,az])
wMeas = Matrix([wx,wy,wz]) 

B_L = Matrix([mx,0,0])
g_L = Matrix([0,0,g])

# Validate quaternion math

In [5]:
print('dcm')
display( quat2dcm(q_toLfromB) )
print('[q]L')
display( skew4Left(q_toLfromB) )
print('[q]R')
display( skew4Right(q_toLfromB) )



dcm


⎡   2     2      2      2                                                    ⎤
⎢q_w  + qₓ  - q_y  - q_z    -2⋅q_w⋅q_z + 2⋅qₓ⋅q_y      2⋅q_w⋅q_y + 2⋅qₓ⋅q_z  ⎥
⎢                                                                            ⎥
⎢                             2     2      2      2                          ⎥
⎢  2⋅q_w⋅q_z + 2⋅qₓ⋅q_y    q_w  - qₓ  + q_y  - q_z    -2⋅q_w⋅qₓ + 2⋅q_y⋅q_z  ⎥
⎢                                                                            ⎥
⎢                                                       2     2      2      2⎥
⎣ -2⋅q_w⋅q_y + 2⋅qₓ⋅q_z      2⋅q_w⋅qₓ + 2⋅q_y⋅q_z    q_w  - qₓ  - q_y  + q_z ⎦

[q]L


⎡q_w  -qₓ   -q_y  -q_z⎤
⎢                     ⎥
⎢qₓ   q_w   -q_z  q_y ⎥
⎢                     ⎥
⎢q_y  q_z   q_w   -qₓ ⎥
⎢                     ⎥
⎣q_z  -q_y   qₓ   q_w ⎦

[q]R


⎡q_w  -qₓ   -q_y  -q_z⎤
⎢                     ⎥
⎢qₓ   q_w   q_z   -q_y⎥
⎢                     ⎥
⎢q_y  -q_z  q_w    qₓ ⎥
⎢                     ⎥
⎣q_z  q_y   -qₓ   q_w ⎦

# Predict

In [6]:
F = eye(4) + t/2*skew4Right(wMeas)
G = eye(4)

print('F')
display(F)
print('G')
display(G)

F


⎡       -T⋅wₓ    -T⋅w_y   -T⋅w_z ⎤
⎢  1    ──────   ───────  ───────⎥
⎢         2         2        2   ⎥
⎢                                ⎥
⎢T⋅wₓ             T⋅w_z   -T⋅w_y ⎥
⎢────      1      ─────   ───────⎥
⎢ 2                 2        2   ⎥
⎢                                ⎥
⎢T⋅w_y  -T⋅w_z             T⋅wₓ  ⎥
⎢─────  ───────     1      ────  ⎥
⎢  2       2                2    ⎥
⎢                                ⎥
⎢T⋅w_z   T⋅w_y   -T⋅wₓ           ⎥
⎢─────   ─────   ──────      1   ⎥
⎣  2       2       2             ⎦

G


⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

# Update

In [7]:
hx_accel = -qRot(quatConj(q_toLfromB), g_L)
hx_mag   =  qRot(quatConj(q_toLfromB), B_L)

dhdx_accel = hx_accel.jacobian(q_toLfromB)
dhdx_mag   = hx_mag.jacobian(q_toLfromB)


print('hx_accel')
display(hx_accel)
print('hx_mag')
display(hx_mag)
print('dhdx_accel')
display(dhdx_accel)
mprint(dhdx_accel)
print('dhdx_mag')
display(dhdx_mag)
mprint(dhdx_mag)

hx_accel


⎡ -g⋅(-2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎤
⎢                             ⎥
⎢  -g⋅(2⋅q_w⋅qₓ + 2⋅q_y⋅q_z)  ⎥
⎢                             ⎥
⎢   ⎛   2     2      2      2⎞⎥
⎣-g⋅⎝q_w  - qₓ  - q_y  + q_z ⎠⎦

hx_mag


⎡   ⎛   2     2      2      2⎞⎤
⎢mₓ⋅⎝q_w  + qₓ  - q_y  - q_z ⎠⎥
⎢                             ⎥
⎢ mₓ⋅(-2⋅q_w⋅q_z + 2⋅qₓ⋅q_y)  ⎥
⎢                             ⎥
⎣  mₓ⋅(2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎦

dhdx_accel


⎡2⋅g⋅q_y   -2⋅g⋅q_z  2⋅g⋅q_w   -2⋅g⋅qₓ ⎤
⎢                                      ⎥
⎢-2⋅g⋅qₓ   -2⋅g⋅q_w  -2⋅g⋅q_z  -2⋅g⋅q_y⎥
⎢                                      ⎥
⎣-2⋅g⋅q_w   2⋅g⋅qₓ   2⋅g⋅q_y   -2⋅g⋅q_z⎦

Matrix([
[ 2*g*q_y, -2*g*q_z,  2*g*q_w, -2*g*q_x],
[-2*g*q_x, -2*g*q_w, -2*g*q_z, -2*g*q_y],
[-2*g*q_w,  2*g*q_x,  2*g*q_y, -2*g*q_z]])
dhdx_mag


⎡2⋅mₓ⋅q_w   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_y  -2⋅mₓ⋅q_z⎤
⎢                                         ⎥
⎢-2⋅mₓ⋅q_z  2⋅mₓ⋅q_y   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_w⎥
⎢                                         ⎥
⎣2⋅mₓ⋅q_y   2⋅mₓ⋅q_z  2⋅mₓ⋅q_w    2⋅mₓ⋅qₓ ⎦

Matrix([
[ 2*m_x*q_w, 2*m_x*q_x, -2*m_x*q_y, -2*m_x*q_z],
[-2*m_x*q_z, 2*m_x*q_y,  2*m_x*q_x, -2*m_x*q_w],
[ 2*m_x*q_y, 2*m_x*q_z,  2*m_x*q_w,  2*m_x*q_x]])


In [8]:
qw,qx,qy,qz,vw,vx,vy,vz = symbols('q_w q_x q_y q_z v_w v_x v_y v_z')
g = symbols('g')
q = Matrix([qw,qx,qy,qz])
v = Matrix([vw,vx,vy,vz])

Istar = Matrix.diag([1,-1,-1,-1])

dRdq = skew4Left(skew4Left(q)*v)*Istar + skew4Right(skew4Left(v)*quatConj(q))






        